Classificador de reviews de jogos da steam.

As reviews são compostas por texto e um label (

In [69]:
import pandas as pd

# Limpando o Dataset

Carregando os dados

In [70]:
df = pd.read_csv("SteamReviews.csv.gz", compression="gzip")


In [103]:
from glob import glob

arquivos = sorted(glob("SteamReviews_parte_*.csv.gz"))
df = pd.concat([pd.read_csv(f) for f in arquivos])

In [104]:
df.describe()

,app_id,review_score,review_votes
count,6.417106e+06,6.417106e+06,6.417106e+06
mean,2.274695e+05,6.394992e-01,1.472446e-01
std,1.260451e+05,7.687918e-01,3.543496e-01
min,1.000000e+01,-1.000000e+00,0.000000e+00
25%,2.018100e+05,1.000000e+00,0.000000e+00
50%,2.391600e+05,1.000000e+00,0.000000e+00
75%,3.056200e+05,1.000000e+00,0.000000e+00
max,5.653400e+05,1.000000e+00,1.000000e+00


Escolhendo uma amostra do dataset

In [105]:
df = df.sample(n = 5000)

In [106]:
df = df[['review_text', 'review_score']]

Checando elementos duplicados

In [107]:
df.duplicated().sum()

799

Removendo duplicatas

In [108]:
df.drop_duplicates(inplace=True)

Renomeando colunas

In [109]:
df.rename(columns={'review_text':'text', 'review_score':'target'}, inplace=True)

In [110]:
df

,text,target
49270,"Easily superior to the original game, simply b...",1
184851,This game was good.,1
199217,Press R to restart simulator,1
301728,10/10 would drink blood from skull of raider t...,1
308528,Early Access Review,-1
...,...,...
126343,One of the best platformers I've ever played. ...,1
275943,This is a sandbox game of sorts with the eleme...,1
165043,"This game spins quite a hearty tale, and if yo...",1
486270,One of the best zombie game to this day. Pros...,1


In [111]:
df.reset_index(drop=True, inplace=True)

## Removendo valores faltantes

In [112]:
missing_values = df.isna().sum()
missing_values

text      2
target    0
dtype: int64

In [113]:
df.dropna(inplace=True)

In [114]:
missing_values = df.isna().sum()
missing_values

text      0
target    0
dtype: int64

## Mudando de rótulos -1 e +1 para 0 e 1

In [115]:
df["target"].unique()

array([ 1, -1], dtype=int64)

In [116]:
df["target"] = df["target"].replace({-1:0, 1: 1})

# Pré Processamento

Imports

In [117]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuário\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuário\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Definindo função de conversão de texto:

Torna mínusculas, tokenizadas, PorterStemmer e sem stopwords e pontuações

In [118]:
def convert_text(text):
  text = text.lower()
  text = nltk.word_tokenize(text)

  x = []
  y = []

  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
      x.append(i)
  for i in x:
    y.append(ps.stem(i))
  return ' '.join(y)

In [119]:
df['converted_text'] = df['text'].apply(convert_text)

In [120]:
df['converted_text']

0       easili superior origin game simpli natur /incl...
1                                               game good
2                                   press r restart simul
3       10/10 would drink blood skull raider say drink...
4                                     earli access review
                              ...                        
4196    one best platform 've ever play mood game rela...
4197    sandbox game sort element game dayz fallout ru...
4198    game spin quit hearti tale think simpl kid gam...
4199    one best zombi game day pro wide select campai...
4200    highli recommend game dont know free play one ...
Name: converted_text, Length: 4199, dtype: object

## Aplicando vetorizador
Contador de palavras

In [121]:
target = df.target

In [122]:
reviews = df.converted_text

In [123]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [124]:
X_train_counts = count_vect.fit_transform(reviews)

In [125]:
X_train_counts

<4199x11672 sparse matrix of type '<class 'numpy.int64'>'
	with 116624 stored elements in Compressed Sparse Row format>

## Transformando em tf-idf
Destacar palavras-chave e reduzir o peso de palavras comuns que aparecem em muitos documentos

In [126]:
from sklearn.feature_extraction.text import TfidfTransformer

In [127]:
tfidf_transformer = TfidfTransformer()

In [128]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Modelo

Multinomial Naive-Bayes

## Classificador

In [129]:
from sklearn.naive_bayes import MultinomialNB

In [130]:
clf = MultinomialNB()

## Teste

In [131]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, target, test_size=0.2, random_state=34)

In [132]:
from sklearn.metrics import accuracy_score


In [133]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8142857142857143